### MobileNet-flowers

In [1]:
#전이 학습(MobileNet)으로 5 Flowers 이미지 분류
#사전 훈련된 이미지 임베딩을 사용해 신경망 구축

import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3
CLASS_NAMES = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
CLASS_NAMES

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [4]:
import tensorflow_hub as hub
import os

os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED' #압축된 모형

def read_and_decode(filename, reshape_dims):
  img = tf.io.read_file(filename)
  img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
  img = tf.image.convert_image_dtype(img, tf.float32)

  return tf.image.resize(img, reshape_dims)

def decode_csv(csv_row):
  record_defaults = ["path", "flower"]
  filename, label_string = tf.io.decode_csv(csv_row, record_defaults)
  img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
  label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))

  return img, label

batch_size = 32
lrate = 0.001
l1 = 0.
l2 = 0.
num_hidden = 16

regularizer = tf.keras.regularizers.l1_l2(l1, l2)

train_dataset = (tf.data.TextLineDataset("c:/workspace3/data/images/flowers/train_set.csv").map(decode_csv)).take(100).batch(batch_size)
eval_dataset = (tf.data.TextLineDataset("c:/workspace3/data/images/flowers/eval_set.csv").map(decode_csv)).take(20).batch(batch_size)

layers = [
    hub.KerasLayer(
        "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
        input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
        trainable=False),

    tf.keras.layers.Dense(num_hidden,
                          kernel_regularizer=regularizer,
                          activation='relu'),

    tf.keras.layers.Dense(len(CLASS_NAMES),
                          kernel_regularizer=regularizer,
                          activation='softmax')
]

model = tf.keras.Sequential(layers)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 16)                20496     
                                                                 
 dense_1 (Dense)             (None, 5)                 85        
                                                                 
Total params: 2,278,565
Trainable params: 20,581
Non-trainable params: 2,257,984
_________________________________________________________________
